In [20]:
from cassandra.cluster import Cluster
from cassandra import ConsistencyLevel
from cassandra.query import SimpleStatement
import json
from collections import defaultdict

# Conexión al clúster de Cassandra con los nombres de host de tus nodos
cluster = Cluster(['127.0.0.1'], port=9042)#, auth_provider=auth_provider)

# Crear una sesión
session = cluster.connect()

# Seleccionar el keyspace
session.set_keyspace('videojuego')

## Lectura hall of fame

In [21]:
def get_hall_of_fame(country: str):
    # Establecer consistencia más fuerte
    # Para asegurar la máxima consistencia usamos ALL. Todas las réplicas tendrán que responder a la query
    # session.default_consistency_level = ConsistencyLevel.ALL

    # Query 
    query = """
        SELECT Mazmorra_id, Nombre_mazmorra, Email, Nombre_usuario, Tiempo, Fecha 
        FROM Hall_of_fame 
        WHERE Pais = %s
        ALLOW FILTERING;
    """
    query = SimpleStatement(query,consistency_level = ConsistencyLevel.ALL)
    
    rows = session.execute(query, (country, ))
    
    # Organizar datos en un diccionario agrupado por Mazmorra_id
    dungeon_dict = defaultdict(lambda: {"dungeon_id": 0, "dungeon_name": "", "Top_5": []})

    for row in rows:
        dungeon_id = row.mazmorra_id
        dungeon_dict[dungeon_id]["dungeon_id"] = dungeon_id
        dungeon_dict[dungeon_id]["dungeon_name"] = row.nombre_mazmorra
        dungeon_dict[dungeon_id]["Top_5"].append({
            "email": row.email,
            "user_name": row.nombre_usuario,
            "time_minutes": row.tiempo,
            "date": row.fecha
        })

    result = json.dumps(list(dungeon_dict.values()), indent=4) 
    return result


In [22]:
# Ejemplo de uso
country = "it_IT"
top_players = get_hall_of_fame(country)
print(top_players)


[
    {
        "dungeon_id": 6,
        "dungeon_name": "Arvaleclock, Dungeon of the Snobbish Scientists",
        "Top_5": [
            {
                "email": "ansaldo75@example.net",
                "user_name": "qgarzoni",
                "time_minutes": 0.20000000298023224,
                "date": "2025-03-21 13:58:10"
            },
            {
                "email": "aldo24@example.com",
                "user_name": "ipastine",
                "time_minutes": 1.0,
                "date": "2021-03-31 17:02:02"
            },
            {
                "email": "amicoscarfoglio@example.net",
                "user_name": "alberto97",
                "time_minutes": 1.0,
                "date": "2022-02-11 18:38:22"
            },
            {
                "email": "annibale86@example.net",
                "user_name": "viscontigioffre",
                "time_minutes": 1.0,
                "date": "2022-07-16 18:52:57"
            },
            {
                "em

In [23]:
data = json.loads(top_players)
for row in data:
    if row['dungeon_id'] == 6:
        for i in row['Top_5']:
            print(i)

{'email': 'ansaldo75@example.net', 'user_name': 'qgarzoni', 'time_minutes': 0.20000000298023224, 'date': '2025-03-21 13:58:10'}
{'email': 'aldo24@example.com', 'user_name': 'ipastine', 'time_minutes': 1.0, 'date': '2021-03-31 17:02:02'}
{'email': 'amicoscarfoglio@example.net', 'user_name': 'alberto97', 'time_minutes': 1.0, 'date': '2022-02-11 18:38:22'}
{'email': 'annibale86@example.net', 'user_name': 'viscontigioffre', 'time_minutes': 1.0, 'date': '2022-07-16 18:52:57'}
{'email': 'armaniannetta@example.org', 'user_name': 'fabiavisintini', 'time_minutes': 1.0, 'date': '2021-01-30 03:37:49'}


## Lectura Statistics

In [24]:
def get_user_statistics(user_email: str, dungeon_id: int):
    # Establecer consistencia más fuerte
    
    # Query 
    query = """
        SELECT Tiempo, Fecha 
        FROM Statistic 
        WHERE Email = %s AND Mazmorra_id = %s;
    """
    query = SimpleStatement(query,consistency_level = ConsistencyLevel.ALL)
    
    rows = session.execute(query, (user_email, dungeon_id))
    
    # converitr a lista de diccionarios
    result = [{"time_minute": row.tiempo, "date": row.fecha} for row in rows]
    
    # Convertir a JSON
    return json.dumps(result, indent=4)


In [25]:
# Ejemplo de uso
user_email = 'ansaldo75@example.net'
dungeon_id = 6
statistics_json = get_user_statistics(user_email, dungeon_id)
print(statistics_json)


[
    {
        "time_minute": 0.20000000298023224,
        "date": "2025-03-21 13:58:10"
    },
    {
        "time_minute": 3.0,
        "date": "2022-05-21 05:29:40"
    },
    {
        "time_minute": 7.0,
        "date": "2022-10-08 16:44:55"
    },
    {
        "time_minute": 8.0,
        "date": "2022-08-20 19:13:44"
    },
    {
        "time_minute": 9.0,
        "date": "2022-09-08 01:48:14"
    },
    {
        "time_minute": 10.0,
        "date": "2022-10-13 07:28:31"
    },
    {
        "time_minute": 10.0,
        "date": "2022-09-02 18:38:06"
    },
    {
        "time_minute": 12.0,
        "date": "2022-07-23 06:23:20"
    }
]


## Lectura Top horde

In [26]:
def get_top_horde(country: str, event_id: int, k: int):
    # Reducimos consistencia para asegurar velocidad en las lecturas

    # Query optimizada
    query = """
        SELECT Email, Nombre_usuario, N_killed 
        FROM Top_horde 
        WHERE Evento_id = %s AND Pais = %s 
        LIMIT %s;
    """
    query = SimpleStatement(query,consistency_level = ConsistencyLevel.LOCAL_ONE)
    
    rows = session.execute(query, (event_id, country, k))
    
    # Convertir a lista de diccionarios
    result = [
        {
            "posicion": idx + 1,  # Asignamos un ranking (opcional)
            "user_name": row.nombre_usuario,
            "email": row.email,
            "n_killed": row.n_killed
        }
        for idx, row in enumerate(rows)
    ]

    # Convertir a JSON
    return json.dumps(result, indent=4)


In [27]:
# Ejemplo de uso
country = "ja_JP"
event_id = 2
k = 5
top_horde_json = get_top_horde(country, event_id, k)
print(top_horde_json)


[
    {
        "posicion": 1,
        "user_name": "ltanaka",
        "email": "ymurakami@example.com",
        "n_killed": 23
    },
    {
        "posicion": 2,
        "user_name": "ukobayashi",
        "email": "jyamashita@example.org",
        "n_killed": 22
    },
    {
        "posicion": 3,
        "user_name": "gfukuda",
        "email": "inoueyosuke@example.com",
        "n_killed": 21
    },
    {
        "posicion": 4,
        "user_name": "yasuhiro13",
        "email": "nakagawanaoto@example.org",
        "n_killed": 21
    },
    {
        "posicion": 5,
        "user_name": "rikakato",
        "email": "rika43@example.org",
        "n_killed": 21
    }
]


## Cerramos conexión

In [28]:
# Cerrar la conexión
cluster.shutdown()